In [4]:
import pandas as pd
import numpy as np
from realgam.quantlib import general_utils as gu
from realgam.quantlib.factor_lib.technicals import indicators_cal as ic
from realgam.quantlib.factor_lib.technicals import alpha_101 as alpha101
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'


In [10]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)

In [5]:
stocks_df['alpha1'] = alpha101.alpha_1(stocks_df)

In [6]:
stocks_df

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,alpha1
date,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.8
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,1.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.8


In [ ]:
(-1 * correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6))

In [13]:
stacked_hist = stocks_df.copy()

In [12]:
n_delta = 2
stacked_hist['log_volume'] = np.log(stacked_hist.volume)
stacked_hist[f'volume_delta{n_delta}'] = stacked_hist.groupby('ticker').log_volume.apply()

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume
date,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0
...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0


In [49]:

stocks_hist['ret'] = stocks_hist.groupby('ticker').closeadj.apply(ic.calc_ret)

In [50]:
stocks_hist['retvol20'] = stocks_hist.groupby('ticker').closeadj.apply(ic.calc_roll_std, n = 20)

In [51]:
stocks_hist['closeadj_sqr'] = stocks_hist.closeadj ** 2
stocks_hist['closeadj_argmax'] = stocks_hist.groupby('ticker').closeadj_sqr.rolling(5).apply(np.argmax, engine='cython', raw=True).values

stocks_hist['retvol20_sqr'] = stocks_hist.retvol20 ** 2
stocks_hist['retvol20_argmax'] = stocks_hist.groupby('ticker').retvol20_sqr.rolling(5).apply(np.argmax, engine='cython', raw=True).values

In [52]:
stocks_hist['tsargmax_withcond'] = np.where(stocks_hist.ret < 0, stocks_hist.retvol20_argmax, stocks_hist.closeadj_argmax) + 1

In [53]:
stocks_hist.loc[stocks_hist.closeadj_argmax.isnull() | stocks_hist.retvol20_argmax.isnull(), 'tsargmax_withcond'] = np.nan

In [55]:
stocks_hist['alpha'] = stocks_hist.tsargmax_withcond / 5

In [44]:
# stocks_hist['alpha'] = stocks_hist.groupby('date').tsargmax_withcond.rank(pct=True).values

In [56]:
stocks_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ret,retvol20,closeadj_sqr,closeadj_argmax,retvol20_sqr,retvol20_argmax,tsargmax_withcond,alpha
date,,,,,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,NaN,579.557476,NaN,NaN,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,-0.007975,NaN,570.349924,NaN,NaN,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.022402,NaN,596.189889,NaN,NaN,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.010812,NaN,609.151761,NaN,NaN,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.026214,NaN,641.507584,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.000000,0.006292,4.840000,3.0,0.000040,3.0,4.0,0.8
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,0.004545,0.006356,4.884100,4.0,0.000040,2.0,5.0,1.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.000000,0.006291,4.884100,3.0,0.000040,1.0,4.0,0.8


In [27]:
s_time_chunk = time.time()
stocks_hist['closeadj_argmax'] = stocks_hist.groupby('ticker').closeadj.rolling(5).apply(np.argmax, engine='cython', raw=True).values
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 18.187129259109497 sec


In [28]:
stocks_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ret,retvol20,op1,op2
date,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,-0.007975,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.022402,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.010812,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.026214,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.000000,0.006292,4.840000,3.0
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,0.004545,0.006356,4.884100,4.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.000000,0.006291,4.884100,3.0
